In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

# Data preprocessing
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Use pre-trained resnet model
model_ft = models.resnet50(pretrained=True)

# Add dropout and fully connected layer at the last layer
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 10)
)

model_ft = model_ft.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Note that we are adding L2 regularization in the optimizer
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)

# Define learning rate scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

100%|██████████| 170498071/170498071 [00:05<00:00, 29894332.44it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 150MB/s]


In [2]:
# Training
num_epochs = 10
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)

    model_ft.train()  # Set model to training mode

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in trainloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward
        with torch.set_grad_enabled(True):
            outputs = model_ft(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward + optimize
            optimizer_ft.zero_grad()
            loss.backward()
            optimizer_ft.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    exp_lr_scheduler.step()

    epoch_loss = running_loss / len(trainset)
    epoch_acc = running_corrects.double() / len(trainset)

    print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

print('Training complete')


Epoch 1/10
----------
Train Loss: 0.9050 Acc: 0.6834
Epoch 2/10
----------
Train Loss: 0.5477 Acc: 0.8121
Epoch 3/10
----------
Train Loss: 0.4871 Acc: 0.8339
Epoch 4/10
----------
Train Loss: 0.4445 Acc: 0.8483
Epoch 5/10
----------
Train Loss: 0.4179 Acc: 0.8553
Epoch 6/10
----------
Train Loss: 0.3934 Acc: 0.8644
Epoch 7/10
----------
Train Loss: 0.3735 Acc: 0.8702
Epoch 8/10
----------
Train Loss: 0.3413 Acc: 0.8818
Epoch 9/10
----------
Train Loss: 0.3245 Acc: 0.8887
Epoch 10/10
----------
Train Loss: 0.3167 Acc: 0.8912
Training complete


In [3]:
# Evaluate the model on the test set
model_ft.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the model on the test images: %d %%' % (correct / total))

Accuracy of the model on the test images: 87 %
